# Parallelization using joblib

In [1]:
import time
import gmspy as gm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

You can check the configuration of this computer in advance

In [2]:
import psutil
print("the number of physical cores:", psutil.cpu_count(logical=False))
print("the number of logical cores:", psutil.cpu_count())
print("CPU frequency:", psutil.cpu_freq())

the number of physical cores: 14
the number of logical cores: 20
CPU frequency: scpufreq(current=2300.0, min=0.0, max=2300.0)


Below is the ground motion data built into gmspy, you can also document the PEER database through [loadPEER](/api/#gmspy.loadPEER), or you can read the data from text using [numpy.loadtxt](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html).

In [3]:
ts, acc = gm.load_gm_examples("Kobe")
dt = ts[1] - ts[0]
# You can also use other acc data

Suppose we have 1000 ground motions:

In [4]:
all_accs = [acc] * 1000

A parallel function needs to be defined, here is the IMs of the ground motions

In [5]:
def get_ims(acc, dt):
    """Compute intensity measures for the given acceleration data."""
    GM = gm.SeismoGM(dt=dt, acc=acc, unit="g")
    ims = GM.get_ims(display_results=False)
    pga = ims["PGA"]
    pgv = ims["PGV"]
    pgd = ims["PGD"]
    ia = ims["Ia"]  # Arias Intensity 
    # elastic spectral acceleration for T=1.0s, index 0 is the spectral acceleration
    sat1 = GM.get_elas_spec(Ts=1.0, damp_ratio=0.05)[0]
    # average spectral acceleration, index 0 is the spectral acceleration
    Tavg = np.arange(0.05, 4.05, 0.05)
    avgsa = GM.get_avgsavd(Tavg=Tavg)[0]
    return pga, pgv, pgd, ia, sat1, avgsa

In [6]:
start_time = time.time()

For more details, please refer to [joblib.Parallel](https://joblib.readthedocs.io/en/latest/generated/joblib.Parallel.html).

``n_jobs=-1`` means using all possible CPU cores.

In [7]:
output = Parallel(n_jobs=-1)(delayed(get_ims)(acc, dt=dt) for acc in all_accs)

In [8]:
end_time = time.time()
print(f"Parallel computation took {end_time - start_time:.2f} seconds")

Parallel computation took 18.98 seconds


Convert to pandas.DataFrame

In [9]:
IMS = pd.DataFrame(output, columns=["PGA", "PGV", "PGD", "IA", "SAT1", "AVGSA"])

IMS.head()

,PGA,PGV,PGD,IA,SAT1,AVGSA
0,0.3447,27.677934,9.693236,1.68744,0.351312,0.155794
1,0.3447,27.677934,9.693236,1.68744,0.351312,0.155794
2,0.3447,27.677934,9.693236,1.68744,0.351312,0.155794
3,0.3447,27.677934,9.693236,1.68744,0.351312,0.155794
4,0.3447,27.677934,9.693236,1.68744,0.351312,0.155794
